In [ ]:
try:
  import brax
except ImportError:
  ! pip install git+https://github.com/ruiqizhang99/brax.git@master
  clear_output()
  import brax

import functools
import time
from typing import Any, Callable, Optional, Tuple, Sequence

from absl import logging
from brax import envs
from brax.envs import wrappers
from brax import jumpy as jp
from brax.io import html
from brax.io import model
from brax.training import acting
from brax.training import gradients
from brax.training import pmap
from brax.training import replay_buffers
from brax.training import types
from brax.training.acme import running_statistics
from brax.training.acme import specs
from brax.training.types import Params
from brax.training.types import PRNGKey, Transition
import flax
import jax
import jax.numpy as jnp
import optax
import tensorflow as tf
import numpy as np
Metrics = types.Metrics
Transition = types.Transition
from datetime import datetime

import os
if 'COLAB_TPU_ADDR' in os.environ:
  from jax.tools import colab_tpu
  colab_tpu.setup_tpu()

In [ ]:
env_name = "walker2d"  # @param ['ant', 'fetch', 'grasp', 'halfcheetah', 'hopper', 'humanoid', 'humanoidstandup', 
                       # 'pusher', 'reacher', 'walker2d', 'grasp', 'ur5e']
env = envs.get_environment(env_name=env_name)
state = env.reset(rng=jp.random_prngkey(seed=0))


In [ ]:
env = wrappers.EpisodeWrapper(env, 1000, 1)
env = wrappers.VmapWrapper(env)
env = wrappers.AutoResetWrapper(env)


def actor_step(env_state: envs.State, action, 
                policy_extras=None, extra_fields: Sequence[str] = ()) -> Tuple[envs.State, Transition]:
    nstate = env.step(env_state, action)
    state_extras = {x: nstate.info[x] for x in extra_fields}
    reward = nstate.reward
    assert reward.shape[0] == 1
    return reward.at[0].get(), (nstate, Transition(
        observation=env_state.obs,
        action=action,
        reward=nstate.reward,
        discount=1-nstate.done,
        next_observation=nstate.obs,
        extras={'policy_extras': policy_extras, 'state_extras': state_extras}))

rew2act_grads_fn = jax.grad(actor_step, argnums=1, has_aux=True)

env_key = jax.random.PRNGKey(0)
env_state = env.reset(env_key)


    